In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from jurity.recommenders import BinaryRecoMetrics, RankingRecoMetrics
from jurity.classification import BinaryClassificationMetrics


# MovieLens Lambda = 10

In [2]:
datasets = [('mkl', 'default'), ('openblas', 'default'), ('mkl', 'perturbed'), ('openblas', 'perturbed')]
expectations = dict()
for backend, option in datasets:
    expectations[(backend, option)] = pickle.load(open(os.path.join(
        'output', f'dar_ml_{backend}_expectations_{option}.pkl'), 'rb'))

In [3]:
responses = pd.read_csv('movielens_responses.csv')
responses.head()


,user id,item id,rated
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1


In [4]:
users = pd.read_csv('movielens_users.csv')

test = users[users['set']=='test']
test.head()

,user id,age,gender,zip code,administrator,artist,doctor,educator,engineer,entertainment,...,other,programmer,retired,salesman,scientist,student,technician,writer,uncategorized,set
660,383,42,0,538.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
661,684,28,0,439.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,test
662,693,43,1,615.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
663,417,27,1,361.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,test
664,358,40,0,95.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test


In [5]:
# test = test.merge(responses, how='left', on='user id')
test = test[['user id']]
test.head()

,user id
660,383
661,684
662,693
663,417
664,358


In [6]:
test.reset_index(inplace=True, drop=True)

In [7]:
test.shape

(283, 1)

In [8]:
dataframes = dict()
for backend, option in datasets:
    dataframes[(backend, option)] = pd.DataFrame(expectations[(backend, option)])
    dataframes[(backend, option)] = dataframes[(backend, option)].merge(test, how='left', left_index=True, right_index=True)

dataframes[datasets[0]].head()

,1,2,3,4,5,6,7,8,9,10,...,1674,1675,1676,1677,1678,1679,1680,1681,1682,user id
0,-0.118996,-0.192981,-0.083528,0.155721,0.014775,-0.266013,0.032385,-0.018680,0.150240,-0.001470,...,0.086270,-0.130110,0.273628,-0.018621,0.035965,0.071400,0.117534,-0.109186,0.072221,383
1,0.011378,-0.055341,-0.050771,0.064026,-0.019840,0.002746,0.174475,0.047570,0.018857,-0.050656,...,0.037476,-0.123352,0.060006,0.020672,0.004614,-0.045715,-0.071648,-0.148390,-0.045666,684
2,0.077772,-0.337627,-0.331055,-0.192834,0.157321,0.148280,-0.160932,0.093677,0.815449,-0.250894,...,-0.127514,-0.509772,-0.014379,-0.344671,0.161543,0.072189,0.308705,0.207805,0.138587,693
3,-0.081484,0.089704,-0.038575,-0.003973,-0.077842,-0.035052,-0.123576,-0.224077,-0.121249,-0.205770,...,0.212707,-0.010658,-0.036988,0.060127,-0.042435,-0.033830,0.264270,0.127448,-0.040067,417
4,0.072852,-0.110499,-0.044861,-0.020536,-0.116935,0.350510,-0.164725,-0.175353,-0.118189,0.040144,...,0.010943,0.061708,0.106349,0.098414,0.038192,0.095223,0.189281,0.052211,0.058577,358


In [9]:
dataframes[datasets[0]].shape

(283, 1683)

In [10]:
for backend, option in datasets:
    dataframes[(backend, option)] = dataframes[(backend, option)].melt(id_vars=['user id'], var_name='item id', value_name='raw_score')

dataframes[datasets[0]].shape

(476006, 3)

In [11]:
dataframes[datasets[0]].head()

,user id,item id,raw_score
0,383,1,-0.118996
1,684,1,0.011378
2,693,1,0.077772
3,417,1,-0.081484
4,358,1,0.072852


In [12]:
test = users[users['set']=='test']
test = test.merge(responses, how='left', on='user id')
test = test[['user id', 'item id', 'rated']]
test.head()

,user id,item id,rated
0,383,1,0
1,383,2,0
2,383,3,0
3,383,4,0
4,383,5,0


In [13]:
for backend, option in datasets:
    dataframes[(backend, option)] = dataframes[(backend, option)].merge(test, how='left', on=['user id', 'item id'])

dataframes[datasets[0]].head()

,user id,item id,raw_score,rated
0,383,1,-0.118996,0
1,684,1,0.011378,1
2,693,1,0.077772,0
3,417,1,-0.081484,1
4,358,1,0.072852,0


In [14]:
dataframes[datasets[0]].shape

(476006, 4)

In [15]:
for backend, option in datasets:
    dataframes[(backend, option)]['probability'] = dataframes[(backend, option)].apply(lambda x: 1/(1 + np.exp(-x['raw_score'])), axis=1)
    dataframes[(backend, option)]['pred'] = dataframes[(backend, option)]['probability'] >= 0.5

Accuracy, Precision, Recall, and AUC

In [16]:
for (backend, option), df in dataframes.items():
    acc = BinaryClassificationMetrics.Accuracy().get_score(df['pred'].tolist(), df['rated'].tolist())
    f1 = BinaryClassificationMetrics.F1().get_score(df['pred'].tolist(), df['rated'].tolist())
    precision = BinaryClassificationMetrics.Precision().get_score(df['pred'].tolist(), df['rated'].tolist())
    recall = BinaryClassificationMetrics.Recall().get_score(df['pred'].tolist(), df['rated'].tolist())
    auc = BinaryClassificationMetrics.AUC().get_score(df['pred'].tolist(), df['rated'].tolist())
    print(f'{backend}_{option}:', acc, f1, precision, recall, auc)

mkl_default: 0.5041995268967198 0.12784277784758202 0.5432304261800823 0.07244604346678842 0.5055716807372257
openblas_default: 0.501695356781217 0.12608596334804617 0.5373889011023523 0.07142171410444241 0.5045598561174798
mkl_perturbed: 0.5035419721600148 0.1273471859617508 0.5415344995446123 0.07215791632142349 0.5052872064184191
openblas_perturbed: 0.5035419721600148 0.1273471859617508 0.5415344995446123 0.07215791632142349 0.5052872064184191


# Table 3, NDCG, Precision, Recall Columns

In [17]:
for (backend, option), df in dataframes.items():
    name = f'{backend}_{option}'
    actual = df[['user id', 'item id', 'rated']].drop_duplicates(subset=['user id', 'item id'])
    predicted = df[['user id', 'item id', 'probability']].drop_duplicates(subset=['user id', 'item id'])
    predicted.columns = actual.columns
    
    user_id_column = 'user id'
    item_id_column = 'item id'
    click_column = 'rated'
    ctr = BinaryRecoMetrics.CTR(user_id_column=user_id_column, item_id_column=item_id_column, 
                                click_column=click_column, k=5).get_score(actual, predicted)
    ctr5 = BinaryRecoMetrics.CTR(user_id_column=user_id_column, item_id_column=item_id_column, 
                                 click_column=click_column, k=10).get_score(actual, predicted)
    ctr10 = BinaryRecoMetrics.CTR(user_id_column=user_id_column, item_id_column=item_id_column, 
                                  click_column=click_column, k=25).get_score(actual, predicted)
    ndcg = RankingRecoMetrics.NDCG(user_id_column=user_id_column, item_id_column=item_id_column, 
                                   click_column=click_column, k=5).get_score(actual, predicted)
    ndcg5 = RankingRecoMetrics.NDCG(user_id_column=user_id_column, item_id_column=item_id_column, 
                                    click_column=click_column, k=10).get_score(actual, predicted)
    ndcg10 = RankingRecoMetrics.NDCG(user_id_column=user_id_column, item_id_column=item_id_column, 
                                     click_column=click_column, k=25).get_score(actual, predicted)
    precision = RankingRecoMetrics.Precision(user_id_column=user_id_column, item_id_column=item_id_column, 
                                             click_column=click_column, k=5).get_score(actual, predicted)
    precision5 = RankingRecoMetrics.Precision(user_id_column=user_id_column, item_id_column=item_id_column, 
                                              click_column=click_column, k=10).get_score(actual, predicted)
    precision10 = RankingRecoMetrics.Precision(user_id_column=user_id_column, item_id_column=item_id_column, 
                                               click_column=click_column, k=25).get_score(actual, predicted)
    recall = RankingRecoMetrics.Recall(user_id_column=user_id_column, item_id_column=item_id_column, 
                                       click_column=click_column, k=5).get_score(actual, predicted)
    recall5 = RankingRecoMetrics.Recall(user_id_column=user_id_column, item_id_column=item_id_column, 
                                        click_column=click_column, k=10).get_score(actual, predicted)
    recall10 = RankingRecoMetrics.Recall(user_id_column=user_id_column, item_id_column=item_id_column, 
                                         click_column=click_column, k=25).get_score(actual, predicted)
    
    cumul_reward = df.sort_values('probability', ascending=False).groupby('user id').head(1)['rated'].sum()

    print(name, ' CTR:', ctr, ctr5, ctr10)
    print(name, ' NDCG:', ndcg, ndcg5, ndcg10)
    print(name, ' Precision:', precision, precision5, precision10)
    print(name, ' Recall:', recall, recall5, recall10)
    print(name, ' Cumulative Reward:', cumul_reward)

mkl_default  CTR: 0.14487632508833923 0.12932862190812722 0.11236749116607773
mkl_default  NDCG: 0.0183477283502026 0.025794187088033826 0.04054599224386034
mkl_default  Precision: 0.14487632508833923 0.1293286219081272 0.11236749116607773
mkl_default  Recall: 0.00678601301822198 0.011952470250550193 0.025455490286241087
mkl_default  Cumulative Reward: 50
openblas_default  CTR: 0.13992932862190813 0.13003533568904593 0.11745583038869258
openblas_default  NDCG: 0.020055892236522488 0.02861634089516421 0.0456816767701652
openblas_default  Precision: 0.1399293286219081 0.13003533568904593 0.11745583038869259
openblas_default  Recall: 0.006976173533242761 0.01326671783566324 0.029730586067924853
openblas_default  Cumulative Reward: 54
mkl_perturbed  CTR: 0.1321554770318021 0.11978798586572438 0.11222614840989399
mkl_perturbed  NDCG: 0.01816402366114645 0.02564287186640868 0.04143385906604007
mkl_perturbed  Precision: 0.1321554770318021 0.11978798586572438 0.11222614840989399
mkl_perturbed 

# Table 2, Score, Probability, and Prediction Comparisons

In [18]:
results = []
env = ['MKL', 'OpenBLAS', 'MKL No Perturb', 'OpenBLAS No Perturb']

for (backend1, option1), df1 in dataframes.items():
    name1 = f'{backend1}_{option1}'
    for (backend2, option2), df2 in dataframes.items():
        name2 = f'{backend2}_{option2}'
        if name1 != name2:
            sim1 = np.isclose(df1['raw_score'], df2['raw_score']).sum()
            sim2 = np.isclose(df1['probability'], df2['probability']).sum()
            sim3 = np.isclose(df1['pred'], df2['pred']).sum()
            results.append((name1, name2, sim1, sim2, sim3))

results

[('mkl_default', 'openblas_default', 4, 59, 329066),
 ('mkl_default', 'mkl_perturbed', 2, 45, 184179),
 ('mkl_default', 'openblas_perturbed', 2, 45, 184179),
 ('openblas_default', 'mkl_default', 4, 59, 329066),
 ('openblas_default', 'mkl_perturbed', 0, 33, 204573),
 ('openblas_default', 'openblas_perturbed', 0, 33, 204573),
 ('mkl_perturbed', 'mkl_default', 2, 45, 184179),
 ('mkl_perturbed', 'openblas_default', 0, 33, 204573),
 ('mkl_perturbed', 'openblas_perturbed', 476006, 476006, 476006),
 ('openblas_perturbed', 'mkl_default', 2, 45, 184179),
 ('openblas_perturbed', 'openblas_default', 0, 33, 204573),
 ('openblas_perturbed', 'mkl_perturbed', 476006, 476006, 476006)]